1. 加载LLM
2. 定义工具
3. Prompt
4. 创建Agent
5. 添加memory

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("API KEY:")

API KEY:········


## 1. 加载LLM

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

## 2. 定义工具

In [6]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """返回单词的长度"""
    return len(word)

get_word_length.invoke("abc")

3

In [7]:
tools = [get_word_length]

## 3. Prompt
+ input：包含用户目标的字符串
+ agent_scratchpad：工具调用和工具输出的信息序列

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a very helpful assistent, but don't know current events"
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

## 4. Bind tools to LLM

In [11]:
llm_with_tools = llm.bind_tools(tools)

## 5. 创建Agent

In [13]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [14]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
agent_executor.invoke({"input": "How many letters in the word alpaca"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'alpaca'}`


6The word "alpaca" has 6 letters.

> Finished chain.


{'input': 'How many letters in the word alpaca',
 'output': 'The word "alpaca" has 6 letters.'}

## 6. 添加memory
+ prompt中加入memory占位符
+ 跟踪历史聊天记录

In [17]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),    
        MessagesPlaceholder(variable_name=MEMORY_KEY),        # memory占位符
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [18]:
# 跟踪聊天记录
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [19]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [20]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
# 跟踪历史聊天记录
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" has 5 letters.

> Finished chain.


> Entering new AgentExecutor chain...
"Educa" is not a commonly used word in English. It seems to be a variation or abbreviation of the word "education."

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='The word "educa" has 5 letters.')],
 'output': '"Educa" is not a commonly used word in English. It seems to be a variation or abbreviation of the word "education."'}